In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset
from scipy import sparse
import os
import copy
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.map_representation.map_api import ArgoverseMap
from skimage.transform import rotate

In [11]:
idx = 2
avl = ArgoverseForecastingLoader('./dataset/train_mini/data')
avl.seq_list = sorted(avl.seq_list)

# print(avl[0])
city = copy.deepcopy(avl[idx].city)
"""TIMESTAMP,TRACK_ID,OBJECT_TYPE,X,Y,CITY_NAME"""
df = copy.deepcopy(avl[idx].seq_df)
# print(df, len(df))

agt_ts = np.sort(np.unique(df['TIMESTAMP'].values))
agt_ti = np.sort(np.unique(df['TRACK_ID'].values))
# print('agt_ts:', agt_ts, len(agt_ts))
# print('agt_ti:', agt_ti, len(agt_ti))

mapping = dict()
for i, ts in enumerate(agt_ts):
    mapping[ts] = i
# print('mapping', mapping)

trajs = np.concatenate((
    df.X.to_numpy().reshape(-1, 1),
    df.Y.to_numpy().reshape(-1, 1)), 1)

steps = [mapping[x] for x in df['TIMESTAMP'].values]
steps = np.asarray(steps, np.int64)

print(steps)
objs = df.groupby(['TRACK_ID', 'OBJECT_TYPE']).groups
keys = list(objs.keys())
obj_type = [x[1] for x in keys]
agt_idx = obj_type.index('AGENT')
idcs = objs[keys[agt_idx]]

agt_traj = trajs[idcs]
agt_step = steps[idcs]
del keys[agt_idx]
ctx_trajs, ctx_steps = [], []
for key in keys:
    idcs = objs[key]
    ctx_trajs.append(trajs[idcs])
    ctx_steps.append(steps[idcs])
data = dict()
data['city'] = city
data['trajs'] = [agt_traj] + ctx_trajs
data['steps'] = [agt_step] + ctx_steps

print(data.keys())

[ 0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2
  2  2  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  8  8  8  8  8  8
  8  8  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11 11 12
 12 12 12 12 12 12 13 13 13 13 13 13 13 14 14 14 14 14 14 14 15 15 15 15
 15 15 15 16 16 16 16 16 16 16 17 17 17 17 17 17 17 18 18 18 18 18 18 18
 19 19 19 19 19 19 19 20 20 20 20 20 20 20 21 21 21 21 21 21 21 22 22 22
 22 22 22 22 23 23 23 23 23 23 23 24 24 24 24 24 24 24 25 25 25 25 25 25
 25 26 26 26 26 26 26 26 27 27 27 27 27 27 27 28 28 28 28 28 28 28 29 29
 29 29 29 29 29 30 30 30 30 30 30 30 31 31 31 31 31 31 31 31 32 32 32 32
 32 32 32 32 33 33 33 33 33 33 33 33 34 34 34 34 34 34 34 34 35 35 35 35
 35 35 35 35 36 36 36 36 36 36 36 36 37 37 37 37 37 37 37 38 38 38 38 38
 38 38 38 39 39 39 39 39 39 39 39 39 40 40 40 40 40 40 40 40 40 41 41 41
 41 41 41 41 41 41 42 42 42 42 42 42 42 42 42 43 43

In [3]:
avl.seq_list

[PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/1.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/10.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/101.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/103.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/104.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/105.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/106.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/107.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/108.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/109.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/110.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/111.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/112.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/113.csv'),
 PosixPath('/home/zzyu/GraphGAN/dataset/train_mini/data/115.csv')

In [4]:
avl[0]

In [5]:
df = copy.deepcopy(avl[0].seq_df)
print(df)

        TIMESTAMP                              TRACK_ID OBJECT_TYPE  \
0    3.159715e+08  00000000-0000-0000-0000-000000000000          AV   
1    3.159715e+08  00000000-0000-0000-0000-000000070303      OTHERS   
2    3.159715e+08  00000000-0000-0000-0000-000000070324       AGENT   
3    3.159715e+08  00000000-0000-0000-0000-000000070377      OTHERS   
4    3.159715e+08  00000000-0000-0000-0000-000000070506      OTHERS   
..            ...                                   ...         ...   
332  3.159715e+08  00000000-0000-0000-0000-000000070303      OTHERS   
333  3.159715e+08  00000000-0000-0000-0000-000000070324       AGENT   
334  3.159715e+08  00000000-0000-0000-0000-000000070500      OTHERS   
335  3.159715e+08  00000000-0000-0000-0000-000000070519      OTHERS   
336  3.159715e+08  00000000-0000-0000-0000-000000070518      OTHERS   

               X            Y CITY_NAME  
0    2661.015638  1145.753052       PIT  
1    2647.091877  1133.773471       PIT  
2    2687.072647  117

In [6]:
print(data['city'])
print(len(data['trajs']))
print(len(data['trajs'][0]))

PIT
15
50


In [7]:
orig = data['trajs'][0][19].copy().astype(np.float32)

pre = data['trajs'][0][18] - orig
theta = np.pi - np.arctan2(pre[1], pre[0])
rot = np.asarray([
    [np.cos(theta), -np.sin(theta)],
    [np.sin(theta), np.cos(theta)]], np.float32)


feats, ctrs, gt_preds, has_preds, traj1, traj2 = [], [], [], [], [], []

for traj, step in zip(data['trajs'], data['steps']):
    # print(traj.shape)

    # if 19 not in step:
        # continue

    print('step1:', step, len(step))
    traj2 = traj.copy().astype(np.float32)
    gt_pred = np.zeros((30, 2), np.float32)
    has_pred = np.zeros(30, np.bool)


    future_mask = np.logical_and(step >= 20, step < 50)
    # print('future_mask:', len(future_mask), future_mask)  # 20False + 30True
    post_step = step[future_mask] - 20
    post_traj = traj[future_mask]
    gt_pred[post_step] = post_traj
    has_pred[post_step] = 1

    # print('post_step:', post_step)
    # print('post_traj:', post_traj, post_step.shape)    # (30,2)
    # print('gt_pred:', gt_pred, gt_pred.size)
 
    obs_mask = step < 20
    # print('obs_mask:', obs_mask, obs_mask.size)   # 20True + 30False
    step = step[obs_mask]   # step = 20
    traj = traj[obs_mask]   # obs_traj = 20
    idcs = step.argsort()
    # print('idcs:', idcs)  # 0-19
    step = step[idcs]
    traj = traj[idcs]

    traj2 = traj2[idcs]

    print('step2', step, len(step))

    for i in range(len(step)):
        if step[i] == 19 - (len(step) - 1) + i:
            break

    # print('i =', i)
    step = step[i:]
    traj = traj[i:]
    print('step3:', step)


    feat = np.zeros((20, 3), np.float32)
    feat[step, :2] = np.matmul(rot, (traj - orig.reshape(-1, 2)).T).T
    feat[step, 2] = 1.0
    # print(feat)


    if feat[-1, 0] < x_min or feat[-1, 0] > x_max or feat[-1, 1] < y_min or feat[-1, 1] > y_max:
        continue

    
    # feat[1:, :2] -= feat[:-1, :2]
    # feat[step[0], :2] = 0
    # feats.append(feat)
    # gt_preds.append(gt_pred)
    # break








step1: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49] 50
step2 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] 20
step3: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


NameError: name 'x_min' is not defined

In [ ]:
x_min, x_max, y_min, y_max = [-100.0, 100.0, -100.0, 100.0]
for traj, step in zip(data['trajs'], data['steps']):
    if 19 not in step:
        continue
    # traj2 = traj.copy().astype(np.float32)


    

    gt_pred = np.zeros((30, 2), np.float32)
    has_pred = np.zeros(30, np.bool)
    future_mask = np.logical_and(step >= 20, step < 50)
    post_step = step[future_mask] - 20
    post_traj = traj[future_mask]
    gt_pred[post_step] = post_traj
    has_pred[post_step] = 1
    # print('future_mask:', future_mask)
    # print('post_traj:', post_traj)
    # print('post_step:', post_step)


    all_traj = traj.copy().astype(np.float32)
    has_traj = np.zeros(50, np.bool)
    all_mask = np.logical_and(step >= 0, step < 50)
    all_traj = all_traj[all_mask]
    all_step = step[all_mask]
    # print(all_traj)
    has_traj[all_step] = 1





    obs_mask = step < 20
    step = step[obs_mask]
    # print('step1:', step)
    traj = traj[obs_mask]
    idcs = step.argsort()
    step = step[idcs]
    # print('step2:', step)

    traj = traj[idcs]
    # traj2 = traj2[idcs]
    for i in range(len(step)):
        if step[i] == 19 - (len(step) - 1) + i:
            break
    step = step[i:]
    traj = traj[i:]
    
    # print('traj2:', traj2, traj2.shape)
    # print('step3:', step)

    feat = np.zeros((20, 3), np.float32)
    feat[step, :2] = np.matmul(rot, (traj - orig.reshape(-1, 2)).T).T
    feat[step, 2] = 1.0
    # traj2[step] = traj2


    traj2 = np.zeros((50,3), np.float32)
    traj2[all_step, :2] = np.matmul(rot, (all_traj - orig.reshape(-1,2)).T).T
    traj2[all_step, 2] = 1
    print('traj2_rot:', traj2, traj2.shape)

    if feat[-1, 0] < x_min or feat[-1, 0] > x_max or feat[-1, 1] < y_min or feat[-1, 1] > y_max:
        continue

    ctrs.append(feat[-1, :2].copy())
    traj1.append((feat).copy())
    feat[1:, :2] -= feat[:-1, :2]
    feat[step[0], :2] = 0
    feats.append(feat)
    gt_preds.append(gt_pred)
    has_preds.append(has_pred)

    # break

traj2_rot: [[-2.9350027e+01  6.9818342e-01  1.0000000e+00]
 [-2.7669277e+01  6.1813718e-01  1.0000000e+00]
 [-2.5916361e+01  6.1841351e-01  1.0000000e+00]
 [-2.4357077e+01  5.5845863e-01  1.0000000e+00]
 [-2.2825306e+01  5.2694958e-01  1.0000000e+00]
 [-2.1702209e+01  4.6983230e-01  1.0000000e+00]
 [-1.9933260e+01  5.3167874e-01  1.0000000e+00]
 [-1.8591227e+01  4.0426558e-01  1.0000000e+00]
 [-1.6941511e+01  4.2175511e-01  1.0000000e+00]
 [-1.5494004e+01  3.2551342e-01  1.0000000e+00]
 [-1.3865626e+01  3.1880724e-01  1.0000000e+00]
 [-1.2298267e+01  3.3116364e-01  1.0000000e+00]
 [-1.0726345e+01  2.0909715e-01  1.0000000e+00]
 [-9.3657990e+00  2.3964110e-01  1.0000000e+00]
 [-7.7486482e+00  1.7809501e-01  1.0000000e+00]
 [-6.1332145e+00  1.8610066e-01  1.0000000e+00]
 [-4.6353884e+00  7.5532913e-02  1.0000000e+00]
 [-2.7170575e+00  6.0446549e-02  1.0000000e+00]
 [-1.5914844e+00 -8.1878483e-05  1.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  1.0000000e+00]
 [ 1.5357370e+00 -9.6273743e-

In [ ]:
gt_preds = np.asarray(gt_preds, np.float32)

print(len(gt_preds))
for i in range(12):
    print(gt_preds[i].shape)


0


IndexError: index 0 is out of bounds for axis 0 with size 0